In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster

In [ ]:
include("../../../../functions/func_map.jl")
include("../../../../functions/func_stat.jl")
include("../../../../functions/func_data.jl")
include("../../../../functions/func_plot.jl")

In [ ]:
data_info_dict = load("/home/chuyu/Notebooks/project_place_cell/figures/chuyu/figure_data_info.jld2")

data_info_all = []
for key in keys(data_info_dict)

    append!(data_info_all, data_info_dict[key])
end

In [ ]:
data_info_all

In [ ]:
analyzer = "chuyu"

In [ ]:
data_info_dict["boundary_morphing"]


## All the data

# Test

In [ ]:
data_info = data_info_dict["boundary_morphing"][1]

experiment_filename_1 = data_info[3]
server_1 = data_info[4]

experiment_filename_2 = data_info[1]
server_2 = data_info[2]

experimenter = data_info[5]
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
chamber_roi_1 = read(orientation_correction_file,"chamber_roi")
countour_array_1 = read(orientation_correction_file,"countour_array")
center_loc_1 = read(orientation_correction_file,"center_loc")
close(orientation_correction_file)


img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;

img_bg_end_1 = img_bg_1[:,:,end]
w = size(img_bg_end_1, 1)
l = size(img_bg_end_1, 2)
# check how well they match
img_bg_end_1[chamber_roi_1.!=0].=NaN

# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_2), "chamber_geometry_$(experiment_filename_2).h5"))
chamber_roi_2 = read(orientation_correction_file,"chamber_roi")
countour_array_2 = read(orientation_correction_file,"countour_array")
center_loc_2 = read(orientation_correction_file,"center_loc")
close(orientation_correction_file)

img_bg_2 = h5open(ds_save_2, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;

img_bg_end_2 = img_bg_2[:,:,end]
w = size(img_bg_end_2, 1)
l = size(img_bg_end_2, 2)

# check how well they match
img_bg_end_2[chamber_roi_2.!=0].=NaN
nr_dots = maximum([size(countour_array_1,1), size(countour_array_2,1)])



x1 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_1, 1)), countour_array_1[:,1])
x2 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_1, 1)), countour_array_1[:,2])

y1 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_2, 1)), countour_array_2[:,1])
y2 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_2, 1)), countour_array_2[:,2]);


x = [[x1[i], x2[i]] for i in 1:nr_dots]
y = [[y1[i], y2[i]] for i in 1:nr_dots];

angle_from_center_1 = angle_from(numpy.array(x), center_loc_1)
angle_from_center_2 = angle_from(numpy.array(y), center_loc_2);
_, start_point_1 = findmin(abs.(angle_from_center_1.+pi))
_, start_point_2 = findmin(abs.(angle_from_center_2.+pi))


nr_roll_1 = -start_point_1
nr_roll_2 = -start_point_2

x1 = numpy.roll(x1, nr_roll_1)
x2 = numpy.roll(x2, nr_roll_1);
y1 = numpy.roll(y1, nr_roll_2)
y2 = numpy.roll(y2, nr_roll_2);


x = [[x1[i], x2[i]] for i in 1:nr_dots]
y = [[y1[i], y2[i]] for i in 1:nr_dots];


figure()
scatter(first.(x), last.(x), c=1:length(x), s =3)
imshow(img_bg_end_1', cmap="gray", vmax=300)
title(experiment_filename_1)

figure()
scatter(first.(y), last.(y), c=1:length(y), s =3)
imshow(img_bg_end_2', cmap="gray", vmax=300)
title(experiment_filename_2)

chamber_dots = findall(chamber_roi_1.==1)
chamber_dots_array = [[chamber_dots[i][1], chamber_dots[i][2]] for i in 1:length(chamber_dots)];

chamber_dots_array_sample = chamber_dots_array[numpy.random.choice(1:length(chamber_dots_array), 1000, replace=false)]
chamber_dots_array_reconstruct = map_dots_from_boundary.(chamber_dots_array_sample, x= x, y =x, k= 2);
chamber_dots_array_map = map_dots_from_boundary.(chamber_dots_array_sample, x= x, y =y, k= 2);

err = norm.(chamber_dots_array_sample .- chamber_dots_array_reconstruct)
figure()
hist(err)



file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
specificity_1 = HDF5.readmmap(file["specificity"])
specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
close(file)


file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
all_files = readdir(file_folder_2)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(file_folder_2, save_file_name)
file = h5open(info_filename, "r")
specificity_2 = HDF5.readmmap(file["specificity"])
specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
valid_roi_2 = HDF5.readmmap(file["valid_neurons"])
place_map_all_2 = HDF5.readmmap(file["place_map_all"])
close(file)



save_file_name = "compare_map_results_original.h5"
compare_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
file = h5open(compare_filename, "r")
tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
close(file)


n_pos = size(place_map_all_1,1);
for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n$(n_pos).h5"))
x_bins = read(for_place_calculation_file,"x_bins")
y_bins = read(for_place_calculation_file,"y_bins")
mask_valid = read(for_place_calculation_file,"mask_valid")
close(for_place_calculation_file)

bin_interval = x_bins[2] - x_bins[1]

mask_pixels = findall(mask_valid)
mask_pixels_array = [[(mask_pixels[i][1] .-0.5)*bin_interval, (mask_pixels[i][2] .-0.5)*bin_interval] for i in 1:length(mask_pixels)]
mask_pixels_array_map = map_dots_from_boundary.(mask_pixels_array, x= x, y =y, k= 2);

figure()
scatter(first.(mask_pixels_array), last.(mask_pixels_array), c=1:length(mask_pixels_array), cmap="Dark2")
scatter(first.(x), last.(x), c=1:length(x))
# axis("square")
title(experiment_filename_1)
figure()
scatter(first.(mask_pixels_array_map), last.(mask_pixels_array_map), c=1:length(mask_pixels_array_map), cmap="Dark2")
scatter(first.(y), last.(y), c=1:length(y))
axis("square")
title(experiment_filename_2)

In [ ]:
function map_dots_from_boundary(chosen_point; x, y, k)
    distance_from_chosen = distance_from(numpy.array(x), chosen_point)
    weights = 1 ./(distance_from_chosen .^(k))
    
    mapped_point = sum(weights .* y)/sum(weights)
    return mapped_point
end

function boundary_morphing_map(example_map, mask_pixels, mask_pixels_array_map, bin_interval)
    mapped_map_sum = zeros(Float32, size(example_map))
    mapped_map_count = zeros(Float32, size(example_map))
    for i in 1:length(mask_pixels_array_map)
        index = mask_pixels_array_map[i]./bin_interval
        value = example_map[mask_pixels[i]]
        mapped_map_sum[floor(Int32, index[1]-1):ceil(Int32, index[1]+1), floor(Int32, index[2]-1):ceil(Int32, index[2]+1)] .+= value
        mapped_map_count[floor(Int32, index[1]-1):ceil(Int32, index[1]+1), floor(Int32, index[2]-1):ceil(Int32, index[2]+1)] .+= 1
    end

    mapped_map = mapped_map_sum./mapped_map_count;
    return mapped_map
end

In [ ]:
which_neuron = numpy.random.choice(1:length(tel_place_cell_index))
mapped_map_all = fill(NaN32, size(place_map_all_1, 1), size(place_map_all_1,2), size(place_map_all_1,3))
for (i_neuron, which_neuron) in enumerate(tel_place_cell_index)
    example_map = place_map_all_1[:,:,which_neuron]
    mapped_map_all[:,:,which_neuron] = boundary_morphing_map(example_map, mask_pixels, mask_pixels_array_map, bin_interval)
end


corr_trials_all = fill(NaN32, size(place_map_all_1,3))

for (i_neuron, which_neuron) in enumerate(tel_place_cell_index)
    map_1 = place_map_all_2[:,:, which_neuron]
    map_2 = mapped_map_all[:,:, which_neuron]
    corr_trials_all[which_neuron] = corr_2d(map_1, map_2)
end

# Matching angle

In [ ]:
morphing_need_data = [data_info_dict["boundary_morphing"];data_info_dict["hedgehog_circle"];data_info_dict["corner_cue_circle"]]

@showprogress for which_data = 1:length(morphing_need_data)
    data_info = morphing_need_data[which_data]

    experiment_filename_1 = data_info[3]
    server_1 = data_info[4]

    experiment_filename_2 = data_info[1]
    server_2 = data_info[2]

    experimenter = data_info[5]
    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    # orientation-corrected background image and chamber roi image
    orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
    chamber_roi_1 = read(orientation_correction_file,"chamber_roi")
    countour_array_1 = read(orientation_correction_file,"countour_array")
    center_loc_1 = read(orientation_correction_file,"center_loc")
    close(orientation_correction_file)


    img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
        read(file, "img_bg")
    end;

    img_bg_end_1 = img_bg_1[:,:,end]
    w = size(img_bg_end_1, 1)
    l = size(img_bg_end_1, 2)
    # check how well they match
    img_bg_end_1[chamber_roi_1.!=0].=NaN

    # orientation-corrected background image and chamber roi image
    orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_2), "chamber_geometry_$(experiment_filename_2).h5"))
    chamber_roi_2 = read(orientation_correction_file,"chamber_roi")
    countour_array_2 = read(orientation_correction_file,"countour_array")
    center_loc_2 = read(orientation_correction_file,"center_loc")
    close(orientation_correction_file)

    img_bg_2 = h5open(ds_save_2, "behavior.h5"; raw = true) do file
        read(file, "img_bg")
    end;

    img_bg_end_2 = img_bg_2[:,:,end]
    w = size(img_bg_end_2, 1)
    l = size(img_bg_end_2, 2)

    # check how well they match
    img_bg_end_2[chamber_roi_2.!=0].=NaN
    nr_dots = maximum([size(countour_array_1,1), size(countour_array_2,1)])



    x1 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_1, 1)), countour_array_1[:,1])
    x2 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_1, 1)), countour_array_1[:,2])

    y1 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_2, 1)), countour_array_2[:,1])
    y2 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_2, 1)), countour_array_2[:,2]);


    x = [[x1[i], x2[i]] for i in 1:nr_dots]
    y = [[y1[i], y2[i]] for i in 1:nr_dots];

    angle_from_center_1 = angle_from(numpy.array(x), center_loc_1)
    angle_from_center_2 = angle_from(numpy.array(y), center_loc_2);
    _, start_point_1 = findmin(abs.(angle_from_center_1.+pi))
    _, start_point_2 = findmin(abs.(angle_from_center_2.+pi))


    nr_roll_1 = -start_point_1
    nr_roll_2 = -start_point_2

    x1 = numpy.roll(x1, nr_roll_1)
    x2 = numpy.roll(x2, nr_roll_1);
    y1 = numpy.roll(y1, nr_roll_2)
    y2 = numpy.roll(y2, nr_roll_2);


    x = [[x1[i], x2[i]] for i in 1:nr_dots]
    y = [[y1[i], y2[i]] for i in 1:nr_dots];


    figure()
    scatter(first.(x), last.(x), c=1:length(x), s =3)
    imshow(img_bg_end_1', cmap="gray", vmax=300)
    title(experiment_filename_1)

    figure()
    scatter(first.(y), last.(y), c=1:length(y), s =3)
    imshow(img_bg_end_2', cmap="gray", vmax=300)
    title(experiment_filename_2)

    chamber_dots = findall(chamber_roi_1.==1)
    chamber_dots_array = [[chamber_dots[i][1], chamber_dots[i][2]] for i in 1:length(chamber_dots)];

    chamber_dots_array_sample = chamber_dots_array[numpy.random.choice(1:length(chamber_dots_array), 1000, replace=false)]
    chamber_dots_array_reconstruct = map_dots_from_boundary.(chamber_dots_array_sample, x= x, y =x, k= 2);
    chamber_dots_array_map = map_dots_from_boundary.(chamber_dots_array_sample, x= x, y =y, k= 2);

    err = norm.(chamber_dots_array_sample .- chamber_dots_array_reconstruct)
    figure()
    hist(err)



    file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)


    file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
    all_files = readdir(file_folder_2)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_2, save_file_name)
    file = h5open(info_filename, "r")
    specificity_2 = HDF5.readmmap(file["specificity"])
    specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_2 = HDF5.readmmap(file["valid_neurons"])
    place_map_all_2 = HDF5.readmmap(file["place_map_all"])
    close(file)
    
    
    #### I changed the tel_place_cell_index definition to the union of PC1 and PC2
    info_filename = joinpath(data_path(ds_save_cy_2), "region_roi_bool.h5")
    file = h5open(info_filename, "r")
    whether_tel = HDF5.readmmap(file["whether_tel"])
    close(file) 
    place_cell_index_2 = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.05), findall(whether_tel));
    place_cell_index_1 = intersect(findall(specificity_population_z_2.>3), findall(specificity_shuffle_z_2.>5), findall(specificity_2.>0.05), findall(whether_tel));
    tel_place_cell_index = union(place_cell_index_1,place_cell_index_2)
    
    n_pos = size(place_map_all_1,1);
    for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n$(n_pos).h5"))
    x_bins = read(for_place_calculation_file,"x_bins")
    y_bins = read(for_place_calculation_file,"y_bins")
    mask_valid = read(for_place_calculation_file,"mask_valid")
    close(for_place_calculation_file)

    bin_interval = x_bins[2] - x_bins[1]

    mask_pixels = findall(mask_valid)
    mask_pixels_array = [[(mask_pixels[i][1] .-0.5)*bin_interval, (mask_pixels[i][2] .-0.5)*bin_interval] for i in 1:length(mask_pixels)]
    mask_pixels_array_map = map_dots_from_boundary.(mask_pixels_array, x= x, y =y, k= 2);

    figure()
    scatter(first.(mask_pixels_array), last.(mask_pixels_array), c=1:length(mask_pixels_array), cmap="Dark2")
    scatter(first.(x), last.(x), c=1:length(x))
    # axis("square")
    title(experiment_filename_1)
    figure()
    scatter(first.(mask_pixels_array_map), last.(mask_pixels_array_map), c=1:length(mask_pixels_array_map), cmap="Dark2")
    scatter(first.(y), last.(y), c=1:length(y))
    axis("square")
    title(experiment_filename_2)


    mapped_map_all = fill(NaN32, size(place_map_all_1, 1), size(place_map_all_1,2), size(place_map_all_1,3))
    for (i_neuron, which_neuron) in enumerate(tel_place_cell_index)
        example_map = place_map_all_1[:,:,which_neuron]
        mapped_map_all[:,:,which_neuron] = boundary_morphing_map(example_map, mask_pixels, mask_pixels_array_map, bin_interval)
    end


    corr_trials_all = fill(NaN32, size(place_map_all_1,3))

    for (i_neuron, which_neuron) in enumerate(tel_place_cell_index)
        map_1 = place_map_all_2[:,:, which_neuron]
        map_2 = mapped_map_all[:,:, which_neuron]
        corr_trials_all[which_neuron] = corr_2d(map_1, map_2)
    end

    h5open(joinpath(data_path(ds_save_analyzer_1), "boundary_morph_matchangle_v2.h5"), "w") do file
        file["corr_trials_all"] = corr_trials_all
        file["mapped_map_all"] = mapped_map_all 
        file["tel_place_cell_index"] = tel_place_cell_index 
    end;
end